# WIPRO -> EZCAP Reconciliation
Compare membership difference between WIPRO and EZCAP

In [1]:
import os
import pandas as pd
from datetime import date

pd.set_option("display.max_columns", None)

In [2]:
path_wipro = '../../data/wipro/10.18.21_443918 HP Elig V1.xlsx'
sheet_name = '443918 HP Elig V1 - 10-18-2021'
dtype_wipro = {'MBD Part D Eligibilty Start Da':str, 'RACE_CD':str}

path_ezcap = '../../data/csv/HP_Elig_EZCAP_DW.csv'
dtype = {'REP_POP_ID':str}

# WIPRO
df_wipro = pd.read_excel(path_wipro, sheet_name = sheet_name, dtype=dtype_wipro).rename(columns={'Memnbr':'memnbr'})
df_wipro['memnbr'] = df_wipro['memnbr'].str.strip()
df_wipro['Enroll Status'] = df_wipro['Enroll Status'].str.strip()

# EZCAP
df_ezcap = pd.read_csv(path_ezcap, dtype=dtype)

In [3]:
df_wipro.head(3)

,memnbr,REP_POP_ID,PO_NBR,mcd_cat,SNPTYPE,Eff Start Date,Eff End Date,Enroll Status,Enroll Reason,MBD Part D Eligibilty Start Da,DSINFO PART D Eligibilty Start,delig_ind,hospice_ben,medical ben,rx_ben,dental_ben,mh_inp_ben,mh_dn_ben,mh_outer_ben,cd_inp_ben,cd_dn_ben,cd_outer_ben,RACE_CD,RACE_NAME
0,1AA1FX4NA00,14129 HMO,PO_NBR,mcd_cat,7,20200101,20210228,EAPRV,CMSAPRV,20140701,20140701,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,1,WHITE
1,1AA1FX4NA00,14129 HMO,PO_NBR,mcd_cat,12,20210301,99999999,EAPRV,CMSAPRV,20140701,20140701,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,1,WHITE
2,1AA3XY6MY70,14129 HMO,PO_NBR,mcd_cat,7,20200101,20201231,EAPRV,CMSAPRV,NaN,20161001,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,4,ASIAN


In [4]:
df_ezcap.head(3)

,memnbr,REP_POP_ID,PO_NBR,mcd_cat,SNPTYPE,elig_start,elig_end,delig_ind,hospice_ben,medical_ben,rx_ben,dental_ben,mh_inp_ben,mh_dn_ben,mh_outer_ben,cd_inp_ben,cd_dn_ben,cd_outer_ben
0,1A05CC7XY50,14129,NaN,NaN,7,20200101,20201231,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y
1,1A14RA9QG85,14129,NaN,NaN,9,20210901,20991231,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y
2,1A15GQ4GY59,99999,NaN,NaN,11,20210101,20991231,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y


In [5]:
# on WIPRO, not EZCAP
df_wo = df_wipro[['memnbr']].drop_duplicates()\
    .merge(df_ezcap[['memnbr']].drop_duplicates(), on='memnbr', how='left', indicator=True)\
    .query('_merge == "left_only"')

n_total = len(df_wo['memnbr'].unique())
print(f'{n_total} member ID\'s found in WIPRO only.')

32 member ID's found in WIPRO only.


In [6]:
# join back df_wipro for basic info
df_wo = df_wo[['memnbr']].drop_duplicates().merge(df_wipro, on='memnbr', how='inner')
df_wo['reason'] = 'unknown'

df_wo.head(3)

,memnbr,REP_POP_ID,PO_NBR,mcd_cat,SNPTYPE,Eff Start Date,Eff End Date,Enroll Status,Enroll Reason,MBD Part D Eligibilty Start Da,DSINFO PART D Eligibilty Start,delig_ind,hospice_ben,medical ben,rx_ben,dental_ben,mh_inp_ben,mh_dn_ben,mh_outer_ben,cd_inp_ben,cd_dn_ben,cd_outer_ben,RACE_CD,RACE_NAME,reason
0,1A69C39CY44,14129 HMO,PO_NBR,mcd_cat,7,20180101,20181231,EAPRV,CMSAPRV,NaN,20060101,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,NaN,NaN,unknown
1,1A69C39CY44,14378 CSNP,PO_NBR,mcd_cat,5,20190101,20191231,EAPRV,CMSAPRV,NaN,20060101,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,NaN,NaN,unknown
2,1A69C39CY44,14378 CSNP,PO_NBR,mcd_cat,5,20200101,99999999,EAPRV,CMSAPRV,NaN,20060101,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,NaN,NaN,unknown


In [7]:
# Member with max end date being DAPRV -> disenrolled members
daprv_member = df_wo[(df_wo['Eff End Date'] == 99999999) & (df_wo['Enroll Status'] == 'DAPRV')]['memnbr'].unique()
df_wo['reason']  = df_wo.apply(lambda df: 'disenrolled' if df['memnbr'] in daprv_member else df['reason'], axis=1)

# Tally
print(f'{n_total} member ID\'s found in WIPRO only.\n')
print(df_wo.groupby(['reason'])['memnbr'].nunique())

32 member ID's found in WIPRO only.

reason
disenrolled    16
unknown        16
Name: memnbr, dtype: int64


In [8]:
# Member with future effective dates
future_effdt_member = df_wo[(df_wo['reason'] == 'unknown') & (df_wo['Eff Start Date'] >= int(date.today().strftime('%Y%m%d')))]['memnbr'].unique()
df_wo['reason']  = df_wo.apply(lambda df: 'future effective date' if df['memnbr'] in future_effdt_member else df['reason'], axis=1)

# Tally
print(f'{n_total} member ID\'s found in WIPRO only.\n')
print(df_wo.groupby(['reason'])['memnbr'].nunique())

32 member ID's found in WIPRO only.

reason
disenrolled              16
future effective date     7
unknown                   9
Name: memnbr, dtype: int64


In [9]:
df_wo.query('reason == "unknown"')['memnbr'].unique()

array(['1A69C39CY44', '1ED9XQ2UD23', '2C12XA1EK28', '2TK0RE8NK25',
       '5Q43D03HF07', '6R43RN0GT07', '7E76KX6PQ88', '8WM5Y26JX96',
       '9QE7MK3AN32'], dtype=object)

In [10]:
# Member with different MBI in WIPRO
# These are menaully searched on WIPRO platform, then confirm the MBI on WIPRO can be found in ECD.dbo.MEMB_COMPANY_V
# This happens when there's member ID change, where WIPRO report contains the newest member ID
# We need to update WIPRO and EZCAP by passing this list to the eligibility team (Marc, Luis)
list_diff_WIPRO_mbi = ['1A69C39CY44', '1ED9XQ2UD23', '2C12XA1EK28', '6R43RN0GT07', '8WM5Y26JX96']
df_wo['reason']  = df_wo.apply(lambda df: 'different MBI in WIPRO' if df['memnbr'] in list_diff_WIPRO_mbi else df['reason'], axis=1)

# Tally
print(f'{n_total} member ID\'s found in WIPRO only.\n')
print(df_wo.groupby(['reason'])['memnbr'].nunique())

32 member ID's found in WIPRO only.

reason
different MBI in WIPRO     5
disenrolled               16
future effective date      7
unknown                    4
Name: memnbr, dtype: int64


In [11]:
# Member with OPFROMDT = OPTHRUDT
# This is being checked manually in EZCAP SQL server
# Need to build out process to automatically check for this
list_from_equals_thru = ['7E76KX6PQ88']
df_wo['reason']  = df_wo.apply(lambda df: 'OPTHRUDT = OPFROMDT' if df['memnbr'] in list_from_equals_thru else df['reason'], axis=1)

# Tally
print(f'{n_total} member ID\'s found in WIPRO only.\n')
print(df_wo.groupby(['reason'])['memnbr'].nunique())

32 member ID's found in WIPRO only.

reason
OPTHRUDT = OPFROMDT        1
different MBI in WIPRO     5
disenrolled               16
future effective date      7
unknown                    3
Name: memnbr, dtype: int64


In [17]:
# Finally, create list of Member ID's that requires further inspection by different parties
path_export = '../../output/reconciliation'
today_datestring = str(date.today().strftime('%Y%m%d'))
export_file_name = f'MEMBNBR_on_WIPRO_only_{today_datestring}.csv'

df_wo.query('reason != "disenrolled"')\
    .sort_values(by=['reason', 'memnbr', 'Eff Start Date'])\
    .to_csv(os.path.join(path_export, export_file_name), index=False)
print('Export completed.')

Export completed.


In [13]:
df_wo.query('reason == "unknown"')

,memnbr,REP_POP_ID,PO_NBR,mcd_cat,SNPTYPE,Eff Start Date,Eff End Date,Enroll Status,Enroll Reason,MBD Part D Eligibilty Start Da,DSINFO PART D Eligibilty Start,delig_ind,hospice_ben,medical ben,rx_ben,dental_ben,mh_inp_ben,mh_dn_ben,mh_outer_ben,cd_inp_ben,cd_dn_ben,cd_outer_ben,RACE_CD,RACE_NAME,reason
17,2TK0RE8NK25,14129 HMO,PO_NBR,mcd_cat,7,20200601,99999999,EAPRV,CMSAPRV,NaN,20200401,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,NaN,NaN,unknown
27,5Q43D03HF07,14993 HMO,PO_NBR,mcd_cat,3,20211101,99999999,EAPRV,CMSAPRV,NaN,20211101,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,1,WHITE,unknown
47,9QE7MK3AN32,14993 HMO,PO_NBR,mcd_cat,3,20211101,99999999,EAPRV,CMSAPRV,NaN,20140501,P,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,4,ASIAN,unknown


In [14]:
df_wo.query('reason == "different MBI in WIPRO"')['memnbr'].unique()

array(['1A69C39CY44', '1ED9XQ2UD23', '2C12XA1EK28', '6R43RN0GT07',
       '8WM5Y26JX96'], dtype=object)